In [2]:
import pandas as pd
import numpy as np
import os
import re
import pickle
import torch

from tqdm import tqdm

In [5]:
assert torch.cuda.is_available()
device = "cuda"

In [7]:
#word pieces combination based on tokens ids
def combine_subwords(ids, words):
    
    """The function takes in two arguments both obtained with hf TokenizerFast class:\
    ids: a list of successive non-single ids \
    word_pieces: a list of word pieces
    
    return:
    a dictionary mapping the ids to their respective subwords
    a list of the reconstructed words"""
    
    words = list(map(lambda x:x.replace("Ġ", "").strip(), words))
    
    # Ensure both input lists have the same length
    if len(ids) != len(words):
        raise ValueError("Input lists must have the same length")

    # Create a dictionary to store word pieces by id
    id_word_dict = {}

    # Iterate through the lists and populate the dictionary
    for id, word in zip(ids, words):
        if id not in id_word_dict:
            id_word_dict[id] = []
        id_word_dict[id].append(word)

    # Create the list of tuples by joining the word pieces
    result = [(id, ''.join(word_pieces)) for id, word_pieces in id_word_dict.items() if not id == None]

    #get rid of None key if any. If present is for the special tokens /s\s
    if id_word_dict.get(None):
      del id_word_dict[None]

    return id_word_dict, result

#word pieces combination based on tokens ids
def combine_subembeddings(ids, embeddings, device = None):

    # Ensure both input lists have the same length
    if len(ids) != len(embeddings):
        raise ValueError("Input lists must have the same length")

    # Create a dictionary to store embedding of word pieces by id
    id_emb_dict = {}

    # Iterate through the lists and populate the dictionary
    for id, sub_emb in zip(ids, embeddings):
        if id not in id_emb_dict:
            id_emb_dict[id] = []
        if device:
          id_emb_dict[id].append(sub_emb.cpu().numpy().astype(float))

        else:
          id_emb_dict[id].append(sub_emb.numpy().astype(float))

    # Create the list of tuples by averaging embedding pieces
    result = [(id, np.mean(sub_emb, axis = 0)) for id, sub_emb in id_emb_dict.items() if not id == None]

    #get rid of None key if any. If present is for the special tokens /s\s
    if id_emb_dict.get(None):
      del id_emb_dict[None]

    return id_emb_dict, result

# helper function to extract representations with a given model
def feature_extractor(sent, token, tokenizer, model, device = None):    #token
    tokenized_sent = tokenizer(sent, return_tensors = "pt", truncation = True)
    word_ids = tokenized_sent.word_ids()
    #dynamically get the target token id
    _, combined_words = combine_subwords(word_ids, tokenized_sent.tokens())
    combined_words = [i[1] for i in combined_words]
    #ensure to get both lower cased and non-lower cased tokens (different between tokenizers)
    try:
      tokid = combined_words.index(token.lower())
    except:
      tokid = combined_words.index(token)
    
    #insert code for the gpu
    if device:
      with torch.no_grad():
        output = model(**tokenized_sent.to(device))
    else:
      with torch.no_grad():
          output = model(**tokenized_sent)
    embeddings = output["last_hidden_state"][0,:]
    embs_dict, encoded_sent_fw = combine_subembeddings(word_ids, embeddings, device = device)
    
    return  embs_dict, encoded_sent_fw, tokid

#helper function to select the target embeddings
def extract_target_embs(encoded_sent_fw, tokid, embs_dict):
    target= encoded_sent_fw[tokid][1]
    target_sub_embs = embs_dict[tokid]

    return target, target_sub_embs

#main function to loop over all the sentences and get the target representations
def get_target_embeddings(sents, tokens, sent_ids, tokenizer, model, device = None):  
    if device:
      device = device
    target_embeddings = {}
    total_sub_embs = {}
    #loop over the sentences to extract each representation
    for i in tqdm(range(len(sents))):
        
        sent_id = str(sent_ids[i])
        token = tokens[i]
        sent = sents[i]
        
        #extract the features for the whole sentence
        embs_dict, encoded_sent_fw, target_tokid = feature_extractor(sent, token, tokenizer, model, device =device)   #token
    
        #extract the target embeddings from the given sentence
        target, target_sub_embs = extract_target_embs(encoded_sent_fw, target_tokid, embs_dict)
        #join the token and sent id to create a key for the dict
        key = token +"."+sent_id
        #add value to the key
        target_embeddings[key] = target
        #store the sub embs in a dictionary with k=word.semt_id: [e1...en]
        total_sub_embs[key] = target_sub_embs
  
    return target_embeddings, total_sub_embs

In [13]:
#helper func to write the results
def serialize_embs(embs, file_name:str, model_ckp:str):
    #write the output in the dedicated directory
    output_path = "..\\Data\\Extracted_Embeddings"

    if "/" in model_ckp:
        model_ckp = model_ckp.split("/")[-1]

    with open(os.path.join(output_path, file_name)+model_ckp.split("/")[-1]+".pkl", "wb") as outfile:
        pickle.dump(embs, outfile)
    return print("Done")

In [12]:
with open("Data/complete_df.pkl", "rb") as infile:
    complete_df = pickle.load(infile)
#extract the necessary data and metadata
full_sents = complete_df["Sent._x"].tolist()
ids2extract = complete_df["hf_tnzd_ids"].tolist()
tokens = complete_df["Token"].tolist()
sent_ids = complete_df.index.tolist()

## BabyBERTa

In [14]:
from transformers import RobertaModel, RobertaTokenizerFast
#Model initialization
model_ckp = "phueb/BabyBERTa-2"
tokenizer = RobertaTokenizerFast.from_pretrained(model_ckp, add_prefix_space = True, truncation = True, max_length = 130)
model = RobertaModel.from_pretrained(model_ckp)
#set the model max_length
tokenizer.model_max_length= 128

#extract the embeddings for each word, by averaging when needed and
#separately store the sub-word embeddings for each token
target_embeddings, total_sub_embs = get_target_embeddings(full_sents, tokens, sent_ids,tokenizer,model)

#serialize the target embeddings
serialize_embs(target_embeddings, "target_embeddings_", model_ckp)
serialize_embs(total_sub_embs, "total_subembs_",model_ckp)

Some weights of RobertaModel were not initialized from the model checkpoint at phueb/BabyBERTa-2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 1955/1955 [00:47<00:00, 41.59it/s]

Done
Done


## GPT-2

In [ ]:
from transformers import GPT2TokenizerFast, GPT2Model
model_ckp = 'gpt2-large'
tokenizer = GPT2TokenizerFast.from_pretrained(model_ckp, return_tensors = "pt")
model = GPT2Model.from_pretrained(model_ckp, device_map = "auto")
target_embeddings, total_sub_embs = get_target_embeddings(full_sents, tokens, sent_ids,tokenizer,model, device = device)

#serialize the target embeddings
serialize_embs(target_embeddings, "target_embeddings_", model_ckp)
serialize_embs(total_sub_embs, "total_subembs_",model_ckp)

## Pythia 70M

In [ ]:
from transformers import AutoModel, AutoTokenizer
model_ckp = "EleutherAI/pythia-70m-deduped"

model = AutoModel.from_pretrained(model_ckp, revision="step3000", cache_dir="./pythia-1b-deduped/step3000")
tokenizer = AutoTokenizer.from_pretrained(model_ckp, revision="step3000", cache_dir="./pythia-70m-deduped/step3000")

#extract the embeddings for each word, by averaging when needed and
#separately store the sub-word embeddings for each token
target_embeddings, total_sub_embs = get_target_embeddings(full_sents, tokens, sent_ids,tokenizer,model)

#serialize the extracted embeddings
serialize_embs(target_embeddings, "target_embeddings_", model_ckp)
serialize_embs(total_sub_embs, "total_subembs_",model_ckp)

## GPT-2 XL

In [ ]:
from transformers import GPT2TokenizerFast, GPT2Model
model_ckp = 'gpt2-xl'
tokenizer = GPT2TokenizerFast.from_pretrained(model_ckp, return_tensors = "pt")
model = GPT2Model.from_pretrained(model_ckp, device_map = "auto")
target_embeddings, total_sub_embs = get_target_embeddings(full_sents, tokens, sent_ids,tokenizer,model, device = device)

#serialize the target embeddings
serialize_embs(target_embeddings, "target_embeddings_", model_ckp)
serialize_embs(total_sub_embs, "total_subembs_",model_ckp)